In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler

from datetime import datetime

In [2]:
#pyspark init
builder = SparkSession.Builder().appName('taxi_duration_highAPI')
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/01 22:10:21 WARN Utils: Your hostname, DESKTOP-0H87CFM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/01 22:10:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 22:10:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Read input file (train.csv only, since the test.csv does not have label for each data point)
raw_data = spark.read.csv('train.csv', header=True, inferSchema=True)

In [4]:
#Cast the pickup and dropoff string values into timestamps.
casted_data = raw_data.withColumns({
                        'pickup_datetime' : raw_data['pickup_datetime'].cast('timestamp'),
                        'dropoff_datetime' : raw_data['dropoff_datetime'].cast('timestamp')
                    })

In [13]:
#Get usable columns from the dataframe
#Also convert pickup and dropoff timestamps into time elements and encode `store_and_fwd_flag` feature into binary values
extracted_df = casted_data.selectExpr(
                'vendor_id',
                'YEAR(pickup_datetime)    AS pickup_year',
                'MONTH(pickup_datetime)   AS pickup_month',
                'DAY(pickup_datetime)     AS pickup_day',
                'HOUR(pickup_datetime)    AS pickup_hour',
                'MINUTE(pickup_datetime)  AS pickup_min',
                'SECOND(pickup_datetime)  AS pickup_sec',
                'YEAR(dropoff_datetime)   AS dropoff_year',
                'MONTH(dropoff_datetime)  AS dropoff_month',
                'DAY(dropoff_datetime)    AS dropoff_day',
                'HOUR(dropoff_datetime)   AS dropoff_hour',
                'MINUTE(dropoff_datetime) AS dropoff_min',
                'SECOND(dropoff_datetime) AS dropoff_sec',
                'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                'CASE WHEN store_and_fwd_flag == "Y" THEN 1 ELSE 0 END AS store_and_fwd_flag',
                'trip_duration'
            )

In [14]:
#Extract features name
cols = [col for col in extracted_df.columns if col != 'trip_duration']

#Use VectorAssembler to transform feature columns into a single vector
assember = VectorAssembler(inputCols=cols, outputCol='features')
data = assember.transform(extracted_df).select('features', 'trip_duration')

In [15]:
#Split data into training and testing sets

###training set proportion parameter:
train_size = 0.8
###

train, test = data.randomSplit([train_size, 1 - train_size], seed=24) #Fixed with seed for reproductivity

In [16]:
estimator = DecisionTreeRegressor(featuresCol='features', labelCol='trip_duration')
model = estimator.fit(train)

25/04/01 22:12:12 WARN MemoryStore: Not enough space to cache rdd_60_5 in memory! (computed 5.7 MiB so far)
25/04/01 22:12:12 WARN BlockManager: Persisting block rdd_60_5 to disk instead.
25/04/01 22:12:12 WARN MemoryStore: Not enough space to cache rdd_60_1 in memory! (computed 13.0 MiB so far)
25/04/01 22:12:12 WARN BlockManager: Persisting block rdd_60_1 to disk instead.
25/04/01 22:12:12 WARN MemoryStore: Not enough space to cache rdd_60_0 in memory! (computed 8.7 MiB so far)
25/04/01 22:12:12 WARN BlockManager: Persisting block rdd_60_0 to disk instead.
25/04/01 22:12:12 WARN MemoryStore: Not enough space to cache rdd_60_3 in memory! (computed 8.7 MiB so far)
25/04/01 22:12:12 WARN BlockManager: Persisting block rdd_60_3 to disk instead.
25/04/01 22:12:12 WARN MemoryStore: Not enough space to cache rdd_60_4 in memory! (computed 13.0 MiB so far)
25/04/01 22:12:12 WARN BlockManager: Persisting block rdd_60_4 to disk instead.
25/04/01 22:12:12 WARN MemoryStore: Not enough space to ca

In [17]:
print(model.toDebugString)

DecisionTreeRegressionModel: uid=DecisionTreeRegressor_6dbeb84112cc, depth=5, numNodes=63, numFeatures=19
  If (feature 14 <= -73.87445068359375)
   If (feature 16 <= -73.91570663452148)
    If (feature 17 <= 40.70650291442871)
     If (feature 15 <= 40.72732925415039)
      If (feature 15 <= 40.6978645324707)
       Predict: 704.3571428571429
      Else (feature 15 > 40.6978645324707)
       Predict: 1092.5150882633588
     Else (feature 15 > 40.72732925415039)
      If (feature 17 <= 40.686012268066406)
       Predict: 2021.467626594511
      Else (feature 17 > 40.686012268066406)
       Predict: 1548.8900373241459
    Else (feature 17 > 40.70650291442871)
     If (feature 0 <= 1.5)
      If (feature 15 <= 40.71365928649902)
       Predict: 1013.0823034446489
      Else (feature 15 > 40.71365928649902)
       Predict: 705.405931345331
     Else (feature 0 > 1.5)
      If (feature 15 <= 40.72732925415039)
       Predict: 1101.0949064323966
      Else (feature 15 > 40.72732925415039)
 

In [18]:
train.show(1)

+--------------------+-------------+
|            features|trip_duration|
+--------------------+-------------+
|[1.0,2016.0,1.0,1...|           87|
+--------------------+-------------+
only showing top 1 row



In [19]:
model.featureImportances

SparseVector(19, {0: 0.0007, 3: 0.9818, 4: 0.0006, 9: 0.0006, 10: 0.0001, 12: 0.0, 13: 0.0, 14: 0.0075, 15: 0.0018, 16: 0.0057, 17: 0.0011})